In [ ]:
PROJECT_ID = 'clean-galaxy-300414'
IMAGE_PREFIX = 'jeongmin-titanic'
PREPROC_DIR = 'preprocess'
TRAIN_DIR = 'train'

WORK_BUCKET = 'gs://clean-galaxy-300414-kubeflowpipelines-default'
RAW_CSV_GCS_URI = WORK_BUCKET + '/rawdata/train.csv'
PREPROC_CSV_GCS_URI = WORK_BUCKET + '/preprocdata/processed_train.csv'
ACC_CSV_GCS_URI = WORK_BUCKET + '/latestacc/accuracy.csv'
MODEL_PKL_GCS_URI = WORK_BUCKET + '/model/model.pkl'
STAGE_GCS_FOLDER = WORK_BUCKET + '/stage'

AIPJOB_TRAINER_GCS_PATH = WORK_BUCKET + '/train/titanic_train.tar.gz'
AIPJOB_OUTPUT_GCS_PATH = WORK_BUCKET + '/train/output/'

In [ ]:
import os
os.environ['PROJECT_ID'] = PROJECT_ID
os.environ['IMAGE_PREFIX'] = IMAGE_PREFIX
os.environ['PREPROC_DIR'] = PREPROC_DIR
os.environ['TRAIN_DIR'] = TRAIN_DIR
os.environ['WORK_BUCKET'] = WORK_BUCKET
os.environ['RAW_CSV_GCS_URI'] = RAW_CSV_GCS_URI
os.environ['PREPROC_CSV_GCS_URI'] = PREPROC_CSV_GCS_URI
os.environ['ACC_CSV_GCS_URI'] = ACC_CSV_GCS_URI
os.environ['MODEL_PKL_GCS_URI'] = MODEL_PKL_GCS_URI
os.environ['STAGE_GCS_FOLDER'] = STAGE_GCS_FOLDER
os.environ['AIPJOB_TRAINER_GCS_PATH'] = AIPJOB_TRAINER_GCS_PATH
os.environ['AIPJOB_OUTPUT_GCS_PATH'] = AIPJOB_OUTPUT_GCS_PATH

In [ ]:
!echo $PROJECT_ID
!echo $IMAGE_PREFIX
!echo $PREPROC_DIR
!echo $TRAIN_DIR
!echo $WORK_BUCKET
!echo $RAW_CSV_GCS_URI
!echo $PREPROC_CSV_GCS_URI
!echo $ACC_CSV_GCS_URI
!echo $MODEL_PKL_GCS_URI
!echo $STAGE_GCS_FOLDER
!echo $AIPJOB_TRAINER_GCS_PATH
!echo $AIPJOB_OUTPUT_GCS_PATH

In [ ]:
!docker build -t $IMAGE_PREFIX-$PREPROC_DIR $PREPROC_DIR/.
!docker tag $IMAGE_PREFIX-$PREPROC_DIR:latest gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$PREPROC_DIR:latest
!docker push gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$PREPROC_DIR:latest

In [ ]:
!docker run gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$PREPROC_DIR:latest --raw_csv_gcs_uri $RAW_CSV_GCS_URI --preproc_csv_gcs_uri $PREPROC_CSV_GCS_URI

In [ ]:
!docker build -t $IMAGE_PREFIX-$TRAIN_DIR $TRAIN_DIR/.
!docker tag $IMAGE_PREFIX-$TRAIN_DIR:latest gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest
!docker push gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest

In [ ]:
!docker run gcr.io/$PROJECT_ID/$IMAGE_PREFIX-$TRAIN_DIR:latest \
    --preproc_csv_gcs_uri $PREPROC_CSV_GCS_URI \
    --model_pkl_gcs_uri $MODEL_PKL_GCS_URI \
    --acc_csv_gcs_uri $ACC_CSV_GCS_URI \
    --min_acc_progress 0.000001

In [ ]:
!rm -fr titanic_train.tar.gz
!tar zcvf titanic_train.tar.gz *
!gsutil cp titanic_train.tar.gz $AIPJOB_TRAINER_GCS_PATH